d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 1200px">
</div>

# Horovod

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
 - Use Horovod to train a distributed neural network
  
HorovodRunner is a general API to run distributed DL workloads on Databricks using Uber’s [Horovod](https://github.com/uber/horovod) framework. By integrating Horovod with Spark’s barrier mode, Databricks is able to provide higher stability for long-running deep learning training jobs on Spark. HorovodRunner takes a Python method that contains DL training code with Horovod hooks. This method gets pickled on the driver and sent to Spark workers. A Horovod MPI job is embedded as a Spark job using barrier execution mode. The first executor collects the IP addresses of all task executors using BarrierTaskContext and triggers a Horovod job using mpirun. Each Python MPI process loads the pickled program back, deserializes it, and runs it.

<br>

![](https://files.training.databricks.com/images/horovod-runner.png)

For additional resources, see:
* [Horovod Runner Docs](https://docs.microsoft.com/en-us/azure/databricks/applications/deep-learning/distributed-training/horovod-runner)
* [Horovod Runner webinar](https://vimeo.com/316872704/e79235f62c)

In [3]:
%run "./Includes/Classroom-Setup"

## Build Model

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
tf.random.set_seed(42)

def build_model():
  return Sequential([Dense(20, input_dim=8, activation="relu"),
                     Dense(20, activation="relu"),
                     Dense(1, activation="linear")]) # Keep the output layer as linear because this is a regression problem

## Shard Data

From the [Horovod docs](https://github.com/horovod/horovod/blob/master/docs/concepts.rst):

Horovod core principles are based on the MPI concepts size, rank, local rank, allreduce, allgather, and broadcast. These are best explained by example. Say we launched a training script on 4 servers, each having 4 GPUs. If we launched one copy of the script per GPU:

* Size would be the number of processes, in this case, 16.

* Rank would be the unique process ID from 0 to 15 (size - 1).

* Local rank would be the unique process ID within the server from 0 to 3.

We need to shard our data across our processes.  **NOTE:** We are using a Pandas DataFrame for demo purposes. In the next notebook we will use Parquet files with Petastorm for better scalability.

In [7]:
from sklearn.datasets.california_housing import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def get_dataset(rank=0, size=1):
  scaler = StandardScaler()
  cal_housing = fetch_california_housing(data_home=f"{working_dir}/{rank}/")
  X_train, X_test, y_train, y_test = train_test_split(cal_housing.data,
                                                      cal_housing.target,
                                                      test_size=0.2,
                                                      random_state=1)
  scaler.fit(X_train)
  X_train = scaler.transform(X_train[rank::size])
  y_train = y_train[rank::size]
  X_test = scaler.transform(X_test[rank::size])
  y_test = y_test[rank::size]
  return (X_train, y_train), (X_test, y_test)

## Horovod

In [9]:
import horovod.tensorflow.keras as hvd
from tensorflow.keras import optimizers
import tensorflow.keras.backend as K

def run_training_horovod():
  # Horovod: initialize Horovod
  hvd.init()
    # If using GPU: pin GPU to be used to process local rank (one GPU per process)
#   gpus = tf.config.experimental.list_physical_devices('GPU')
#   for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
#   if gpus:
#       tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')
  
  print(f"Rank is: {hvd.rank()}")
  print(f"Size is: {hvd.size()}")
  
  (X_train, y_train), (X_test, y_test) = get_dataset(hvd.rank(), hvd.size())
  
  model = build_model()
  
  # Horovod: adjust learning rate based on number of GPUs/CPUs
  optimizer = optimizers.Adam(lr=0.001*hvd.size())
  
  # Horovod: add Horovod Distributed Optimizer
  optimizer = hvd.DistributedOptimizer(optimizer)
  
  # In Tensorflow 2.x specify `experimental_run_tf_function=False` to ensure TensorFlow uses hvd.DistributedOptimizer() to compute gradients.
  # See horovod docs at https://horovod.readthedocs.io/en/latest/tensorflow.html
  model.compile(optimizer=optimizer, loss="mse", metrics=["mse"], experimental_run_tf_function=False)
  
  history = model.fit(X_train, y_train, validation_split=.2, epochs=10, batch_size=64, verbose=2)

Test it out on just the driver.

In [11]:
from sparkdl import HorovodRunner

hr = HorovodRunner(np=-1)
hr.run(run_training_horovod)

HorovodRunner will stream all training logs to notebook cell output. If there are too many logs, you
can adjust the log level in your train method. Or you can set driver_log_verbosity to
'log_callback_only' and use a HorovodRunner log callback on the first worker to get concise
progress updates.
HorovodRunner will launch Horovod jobs on the driver node. There would be resource contention if you
share the cluster with others.
The global names read or written to by the pickled function are {'hvd', 'build_model', 'print', 'get_dataset', 'optimizers'}.
The pickled object size is 2966 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Executing command: ['mpirun', '--allow-run-as-root', '-np', '1', '-H', 'localhost', '--stdin', 'none', '--tag-output', '-mca', 'rmaps', 'seq', '--bind-to', 'none', '-x', 'NCCL_DEBUG=INFO', '-mca', 'pml', 'ob1', '-mca', 'btl', '^openib', '-mca', 'plm_rsh_agent', 'ssh -o StrictHostKeyChecking=no -i /tmp/HorovodRunner_c72c1a05b4e7/id_rsa', 'bash', '/tmp/HorovodRunner_c72c1a05b4e7/launch.sh'].

[1,0]<stdout>:Rank is: 0
[1,0]<stdout>:Size is: 1
[1,0]<stderr>:Downloading Cal. housing from https://ndownloader.figshare.com/files/5976036 to /dbfs/user/odl_user_195841@databrickslabs.onmicrosoft.com/deep_learning/dl_06_horovod/0/
[1,0]<stderr>:Downloading Cal. housing from https://ndownloader.figshare.com/files/5976036 to /dbfs/user/odl_user_195841@databrickslabs.onmicrosoft.com/deep_learning/dl_06_horovod/0/
[1,0]<stderr>:2020-06-22 19:16:35.743381: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
[1,0]<stderr>:2020-06-22 19:16:35.750577: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2294685000 Hz
[1,0]<stderr>:2020-06-22 19:16:35.750967: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55eabdda06e0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
[1,0]<stderr>:2020-06-22 19:16:35.750999: I tensorflow/compiler/xla/service/service.cc:176] StreamExecutor device (0): Host, Default Version
[1,0]<stdout>:Epoch 1/10
[1,0]<stdout>:207/207 - 0s - loss: 2.6824 - mse: 2.6824 - val_loss: 0.8186 - val_mse: 0.8186
[1,0]<stdout>:Epoch 2/10
[1,0]<stdout>:207/207 - 0s - loss: 0.7604 - mse: 0.7604 - val_loss: 0.6136 - val_mse: 0.6136
[1,0]<stdout>:Epoch 3/10
[1,0]<stdout>:207/207 - 0s - loss: 0.5593 - mse: 0.5593 - val_loss: 0.4940 - val_mse: 0.4940
[1,0]<stdout>:Epoch 4/10
[1,0]<stdout>:207/207 - 0s - loss: 0.4559 - mse: 0.4559 - val_loss: 0.4383 - val_mse: 0.4383
[1,0]<stdout>:Epoch 5/10
[1,0]<stdout>:207/207 - 0s - loss: 0.4141 - mse: 0.4141 - val_loss: 0.4185 - val_mse: 0.4185
[1,0]<stdout>:Epoch 6/10
[1,0]<stdout>:207/207 - 0s - loss: 0.3951 - mse: 0.3951 - val_loss: 0.4142 - val_mse: 0.4142
[1,0]<stdout>:Epoch 7/10
[1,0]<stdout>:207/207 - 0s - loss: 0.3837 - mse: 0.3837 - val_loss: 0.3952 - val_mse: 0.3952
[1,0]<stdout>:Epoch 8/10
[1,0]<stdout>:207/207 - 0s - loss: 0.3737 - mse: 0.3737 - val_loss: 0.3898 - val_mse: 0.3898
[1,0]<stdout>:Epoch 9/10
[1,0]<stdout>:207/207 - 0s - loss: 0.3670 - mse: 0.3670 - val_loss: 0.3832 - val_mse: 0.3832
[1,0]<stdout>:Epoch 10/10
[1,0]<stdout>:207/207 - 0s - loss: 0.3623 - mse: 0.3623 - val_loss: 0.3761 - val_mse: 0.3761

# Better Horovod

In [13]:
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import *

def run_training_horovod():
  # Horovod: initialize Horovod.
  hvd.init()
  
#  # If using GPU: pin GPU to be used to process local rank (one GPU per process)
#   gpus = tf.config.experimental.list_physical_devices('GPU')
#   for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
#   if gpus:
#       tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')


  print(f"Rank is: {hvd.rank()}")
  print(f"Size is: {hvd.size()}")
  
  (X_train, y_train), (X_test, y_test) = get_dataset(hvd.rank(), hvd.size())
  
  model = build_model()
  
  # Horovod: adjust learning rate based on number of GPUs.
  optimizer = optimizers.Adam(lr=0.001*hvd.size())
  
  # Horovod: add Horovod Distributed Optimizer.
  optimizer = hvd.DistributedOptimizer(optimizer)

  # In Tensorflow 2.x specify `experimental_run_tf_function=False` to ensure TensorFlow uses hvd.DistributedOptimizer() to compute gradients.
  # See horovod docs at https://horovod.readthedocs.io/en/latest/keras.html
  model.compile(optimizer=optimizer, loss="mse", metrics=["mse"], experimental_run_tf_function=False)

  # Use the optimized FUSE Mount
  checkpoint_dir = f"{working_dir}/horovod_checkpoint_weights.ckpt"
  
  callbacks = [
    # Horovod: broadcast initial variable states from rank 0 to all other processes.
    # This is necessary to ensure consistent initialization of all workers when
    # training is started with random weights or restored from a checkpoint.
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),

    # Horovod: average metrics among workers at the end of every epoch.
    # Note: This callback must be in the list before the ReduceLROnPlateau,
    # TensorBoard or other metrics-based callbacks.
    hvd.callbacks.MetricAverageCallback(),

    # Horovod: using `lr = 1.0 * hvd.size()` from the very beginning leads to worse final
    # accuracy. Scale the learning rate `lr = 1.0` ---> `lr = 1.0 * hvd.size()` during
    # the first five epochs. See https://arxiv.org/abs/1706.02677 for details.
    hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=5, verbose=1),
    
    # Reduce the learning rate if training plateaus.
    ReduceLROnPlateau(patience=10, verbose=1)
  ]
  
  # Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
  if hvd.rank() == 0:
    callbacks.append(ModelCheckpoint(checkpoint_dir, save_weights_only=True))
  
  history = model.fit(X_train, y_train, validation_split=.2, epochs=10, batch_size=64, verbose=2, callbacks=callbacks)

Test it out on just the driver.

In [15]:
hr = HorovodRunner(np=-1)
hr.run(run_training_horovod)

HorovodRunner will stream all training logs to notebook cell output. If there are too many logs, you
can adjust the log level in your train method. Or you can set driver_log_verbosity to
'log_callback_only' and use a HorovodRunner log callback on the first worker to get concise
progress updates.
HorovodRunner will launch Horovod jobs on the driver node. There would be resource contention if you
share the cluster with others.
The global names read or written to by the pickled function are {'ModelCheckpoint', 'hvd', 'build_model', 'ReduceLROnPlateau', 'print', 'working_dir', 'get_dataset', 'optimizers'}.
The pickled object size is 3435 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Executing command: ['mpirun', '--allow-run-as-root', '-np', '1', '-H', 'localhost', '--stdin', 'none', '--tag-output', '-mca', 'rmaps', 'seq', '--bind-to', 'none', '-x', 'NCCL_DEBUG=INFO', '-mca', 'pml', 'ob1', '-mca', 'btl', '^openib', '-mca', 'plm_rsh_agent', 'ssh -o StrictHostKeyChecking=no -i /tmp/HorovodRunner_c21242bdafe8/id_rsa', 'bash', '/tmp/HorovodRunner_c21242bdafe8/launch.sh'].

[1,0]<stdout>:Rank is: 0
[1,0]<stdout>:Size is: 1
[1,0]<stderr>:2020-06-22 19:19:33.431496: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
[1,0]<stderr>:2020-06-22 19:19:33.436231: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2294685000 Hz
[1,0]<stderr>:2020-06-22 19:19:33.436555: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5586bd5a1300 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
[1,0]<stderr>:2020-06-22 19:19:33.436587: I tensorflow/compiler/xla/service/service.cc:176] StreamExecutor device (0): Host, Default Version
[1,0]<stdout>:Epoch 1/10
[1,0]<stderr>:WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.148933). Check your callbacks.
[1,0]<stderr>:Method (on_train_batch_end) is slow compared to the batch update (0.148933). Check your callbacks.
[1,0]<stdout>:207/207 - 1s - loss: 1.7693 - mse: 1.7693 - val_loss: 0.6971 - val_mse: 0.6971 - lr: 0.0010
[1,0]<stdout>:Epoch 2/10
[1,0]<stdout>:207/207 - 1s - loss: 0.6130 - mse: 0.6130 - val_loss: 0.5420 - val_mse: 0.5420 - lr: 0.0010
[1,0]<stdout>:Epoch 3/10
[1,0]<stdout>:207/207 - 1s - loss: 0.4930 - mse: 0.4930 - val_loss: 0.4652 - val_mse: 0.4652 - lr: 0.0010
[1,0]<stdout>:Epoch 4/10
[1,0]<stdout>:207/207 - 1s - loss: 0.4337 - mse: 0.4337 - val_loss: 0.4265 - val_mse: 0.4265 - lr: 0.0010
[1,0]<stdout>:Epoch 5/10
[1,0]<stdout>:
[1,0]<stdout>:Epoch 5: finished gradual learning rate warmup to 0.001.
[1,0]<stdout>:207/207 - 1s - loss: 0.4048 - mse: 0.4048 - val_loss: 0.4128 - val_mse: 0.4128 - lr: 0.0010
[1,0]<stdout>:Epoch 6/10
[1,0]<stdout>:207/207 - 1s - loss: 0.3904 - mse: 0.3904 - val_loss: 0.4015 - val_mse: 0.4015 - lr: 0.0010
[1,0]<stdout>:Epoch 7/10
[1,0]<stdout>:207/207 - 1s - loss: 0.3809 - mse: 0.3809 - val_loss: 0.3904 - val_mse: 0.3904 - lr: 0.0010
[1,0]<stdout>:Epoch 8/10
[1,0]<stdout>:207/207 - 1s - loss: 0.3752 - mse: 0.3752 - val_loss: 0.3849 - val_mse: 0.3849 - lr: 0.0010
[1,0]<stdout>:Epoch 9/10
[1,0]<stdout>:207/207 - 1s - loss: 0.3706 - mse: 0.3706 - val_loss: 0.3831 - val_mse: 0.3831 - lr: 0.0010
[1,0]<stdout>:Epoch 10/10
[1,0]<stdout>:207/207 - 1s - loss: 0.3654 - mse: 0.3654 - val_loss: 0.3822 - val_mse: 0.3822 - lr: 0.0010

## Run on all workers

In [17]:
## OPTIONAL: You can enable Horovod Timeline as follows, but can incur slow down from frequent writes, and have to export out of Databricks to upload to chrome://tracing
# import os
# os.environ["HOROVOD_TIMELINE"] = f"{working_dir}/_timeline.json"

hr = HorovodRunner(np=0)
hr.run(run_training_horovod)


HorovodRunner will stream all training logs to notebook cell output. If there are too many logs, you
can adjust the log level in your train method. Or you can set driver_log_verbosity to
'log_callback_only' and use a HorovodRunner log callback on the first worker to get concise
progress updates.
Setting np=0 is deprecated and it will be removed in the next major Databricks Runtime release.
Choosing np based on the total task slots at runtime is unreliable due to dynamic executor
registration. Please set the number of parallel processes you need explicitly.
The global names read or written to by the pickled function are {'ModelCheckpoint', 'hvd', 'build_model', 'ReduceLROnPlateau', 'print', 'working_dir', 'get_dataset', 'optimizers'}.
The pickled object size is 3437 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Start training.
[1,0]<stdout>:Rank is: 0
[1,0]<stdout>:Size is: 4
[1,3]<stdout>:Rank is: 3
[1,3]<stdout>:Size is: 4
[1,1]<stdout>:Rank is: 1
[1,1]<stdout>:Size is: 4
[1,2]<stdout>:Rank is: 2
[1,2]<stdout>:Size is: 4
[1,3]<stderr>:Downloading Cal. housing from https://ndownloader.figshare.com/files/5976036 to /dbfs/user/odl_user_195841@databrickslabs.onmicrosoft.com/deep_learning/dl_06_horovod/3/
[1,3]<stderr>:Downloading Cal. housing from https://ndownloader.figshare.com/files/5976036 to /dbfs/user/odl_user_195841@databrickslabs.onmicrosoft.com/deep_learning/dl_06_horovod/3/
[1,0]<stderr>:2020-06-22 19:20:48.824932: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
[1,0]<stderr>:2020-06-22 19:20:48.844571: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2294685000 Hz
[1,0]<stderr>:2020-06-22 19:20:48.844976: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5619a1640fd0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
[1,0]<stderr>:2020-06-22 19:20:48.845003: I tensorflow/compiler/xla/service/service.cc:176] StreamExecutor device (0): Host, Default Version
[1,1]<stderr>:Downloading Cal. housing from https://ndownloader.figshare.com/files/5976036 to /dbfs/user/odl_user_195841@databrickslabs.onmicrosoft.com/deep_learning/dl_06_horovod/1/
[1,1]<stderr>:Downloading Cal. housing from https://ndownloader.figshare.com/files/5976036 to /dbfs/user/odl_user_195841@databrickslabs.onmicrosoft.com/deep_learning/dl_06_horovod/1/
[1,2]<stderr>:Downloading Cal. housing from https://ndownloader.figshare.com/files/5976036 to /dbfs/user/odl_user_195841@databrickslabs.onmicrosoft.com/deep_learning/dl_06_horovod/2/
[1,2]<stderr>:Downloading Cal. housing from https://ndownloader.figshare.com/files/5976036 to /dbfs/user/odl_user_195841@databrickslabs.onmicrosoft.com/deep_learning/dl_06_horovod/2/
[1,0]<stdout>:Epoch 1/10[1,0]<stdout>:
[1,3]<stderr>:2020-06-22 19:20:52.968484: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
[1,3]<stderr>:2020-06-22 19:20:52.978562: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2294685000 Hz
[1,3]<stderr>:2020-06-22 19:20:52.978791: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55b4ae2d8550 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
[1,3]<stderr>:2020-06-22 19:20:52.978821: I tensorflow/compiler/xla/service/service.cc:176] StreamExecutor device (0): Host, Default Version
[1,3]<stdout>:Epoch 1/10[1,3]<stdout>:
[1,2]<stderr>:2020-06-22 19:20:53.259615: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that thi

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>